#BackPropagation

##Exo1

In [1]:
from random import seed 
from random import random

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[round(random(),1) for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[round(random(),1) for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

In [2]:
def write(network):
  for i in range(len(network)):
    layer = network[i]
    print('layer : ', i+1, '\n')
    for neuron in layer:
      print(neuron, '\n')

In [3]:
#tirage aléatoire
network = initialize_network(2,3,1)
write(network)

layer :  1 

{'weights': [0.7, 0.4, 0.5]} 

{'weights': [0.2, 0.2, 0.8]} 

{'weights': [0.7, 1.0, 0.0]} 

layer :  2 

{'weights': [0.0, 0.4, 0.2, 0.7]} 



##Exo2

In [4]:
seed(1) #fixer le tirage (1 fois aléatoire)
network = initialize_network(3,2,2)
write(network)

layer :  1 

{'weights': [0.1, 0.8, 0.8, 0.3]} 

{'weights': [0.5, 0.4, 0.7, 0.8]} 

layer :  2 

{'weights': [0.1, 0.0, 0.8]} 

{'weights': [0.4, 0.8, 0.0]} 



##Exo3

In [5]:
def activate(weights, inputs):
  activation = weights[-1]
  for i in range(len(weights)-1):
    activation += weights[i] * inputs[i]
  return activation

In [6]:
weights = [3.0,5.0,4]
inputs = [-1,2]
# 3x(-1) + 5x2 + 4 = 11
activate (weights, inputs) #combinaison lineaire

11.0

##Exo4

In [7]:
from math import exp
#transfer neuron activation
def transfer(activation):
  return 1.0 / (1.0 + exp(-activation))
transfer(0)

0.5

##Exo5

In [8]:
# Forward propagate input to a network output
def forward_propagate ( network , row ):
 inputs = row
 for layer in network :
  new_inputs = []
  for neuron in layer :
    activation = activate(neuron['weights'], inputs)
    neuron ['output'] = transfer(activation) # implement activate and transfer functions on the neuron weight and the given input row data . And assign the returned value to neuron [’ output ’]
    new_inputs . append ( neuron ['output'])
  inputs = new_inputs
 return inputs

##Exo6

In [9]:
#test foward propagation
network = initialize_network(2,1,2)
print('######Before########')
write(network)
row = [1,0]
output = forward_propagate(network, row)
print('######After########')
write(network)
print('######output########')
print('output : ', output, '\n')

######Before########
layer :  1 

{'weights': [0.4, 0.7, 0.2]} 

layer :  2 

{'weights': [0.9, 0.9]} 

{'weights': [0.0, 0.0]} 

######After########
layer :  1 

{'weights': [0.4, 0.7, 0.2], 'output': 0.6456563062257954} 

layer :  2 

{'weights': [0.9, 0.9], 'output': 0.8147372642938806} 

{'weights': [0.0, 0.0], 'output': 0.5} 

######output########
output :  [0.8147372642938806, 0.5] 



##Exo7

In [10]:
#Calculate the derivative of an neuron output
def transfer_derivative(output):
  return output * (1.0 - output)

##Exo8

In [11]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

#Exo9

In [12]:
# test backpropagation of error
expected = [0, 1]
print('######Before#########')
write(network)
backward_propagate_error(network, expected)
print('######After#########')
write(network)

######Before#########
layer :  1 

{'weights': [0.4, 0.7, 0.2], 'output': 0.6456563062257954} 

layer :  2 

{'weights': [0.9, 0.9], 'output': 0.8147372642938806} 

{'weights': [0.0, 0.0], 'output': 0.5} 

######After#########
layer :  1 

{'weights': [0.4, 0.7, 0.2], 'output': 0.6456563062257954, 'delta': -0.025321641068429778} 

layer :  2 

{'weights': [0.9, 0.9], 'output': 0.8147372642938806, 'delta': -0.12297681294192943} 

{'weights': [0.0, 0.0], 'output': 0.5, 'delta': 0.125} 



##Exo10

In [15]:
# Update network weights with error
def update_weights(network, row, l_rate):
  for i in range(len(network)):
    inputs = row[:1]
    if i != 0:
      inputs = [neuron['output']for neuron in network[i - 1]]
    for neuron in network[i]:
      for j in range(len(inputs)):
        neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
      neuron['weights'][-1] += l_rate * neuron['delta']

##Exo11

In [17]:
#Train a network for a fixed number of epoch
def train_network(network, train, l_rate, n_epoch, n_outputs):
  for epoch in range(n_epoch):
    sum_error = 0
    for row in train:
      outputs = forward_propagate(network, row)
      expected = [0 for i in range(n_outputs)]
      expected[row[-1]] = 1
      sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
      backward_propagate_error(network, expected)
      update_weights(network, row, l_rate)
    print('>epoch=%d, lrate=%.3f, error=%.3f'%(epoch, l_rate, sum_error))

##Exo12

In [18]:
# Test training backprop algorithm
# speed(1)
dataset=[[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
n_inputs = len(dataset[0]) - 1
print(n_inputs)
n_outputs = len(set([row[-1]for row in dataset]))

print(n_outputs)
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 100, n_outputs)

2
2
>epoch=0, lrate=0.500, error=5.637
>epoch=1, lrate=0.500, error=5.306
>epoch=2, lrate=0.500, error=5.279
>epoch=3, lrate=0.500, error=5.295
>epoch=4, lrate=0.500, error=5.276
>epoch=5, lrate=0.500, error=5.188
>epoch=6, lrate=0.500, error=5.050
>epoch=7, lrate=0.500, error=4.913
>epoch=8, lrate=0.500, error=4.785
>epoch=9, lrate=0.500, error=4.660
>epoch=10, lrate=0.500, error=4.531
>epoch=11, lrate=0.500, error=4.400
>epoch=12, lrate=0.500, error=4.265
>epoch=13, lrate=0.500, error=4.126
>epoch=14, lrate=0.500, error=3.984
>epoch=15, lrate=0.500, error=3.838
>epoch=16, lrate=0.500, error=3.690
>epoch=17, lrate=0.500, error=3.539
>epoch=18, lrate=0.500, error=3.387
>epoch=19, lrate=0.500, error=3.235
>epoch=20, lrate=0.500, error=3.083
>epoch=21, lrate=0.500, error=2.933
>epoch=22, lrate=0.500, error=2.785
>epoch=23, lrate=0.500, error=2.640
>epoch=24, lrate=0.500, error=2.500
>epoch=25, lrate=0.500, error=2.364
>epoch=26, lrate=0.500, error=2.234
>epoch=27, lrate=0.500, error=2.11

##Exo13

In [20]:
def write(network):
  for i in range(len(network)):
    layer = network[i]
    print('layer :',i+1,'\n')
    for neuron in layer:
	    print(neuron,'\n')
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))
for row in dataset:
	prediction = predict(network, row)
	print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1


###change init net avec ce code et refaire tourner

In [21]:
from random import random
from random import seed
seed(1)
# Initialize a network
nbr_hidden_neuron = [3,5,4]

def initialize_network_multiple_hidden(n_inputs, nbr_hidden_neuron, n_outputs):
# n_inputs = nbr d'inputs

# nbr_hidden_neuron = tableau de taille(n_hidden_layer) qui donne le nombre de neuronne par layer
# n_outputs = nombre d'outputs demandé
	network=list()
	for num_hidden in range(len(nbr_hidden_neuron)): # new! : this is where the number of layer is added 
		nbre_nn = n_inputs if num_hidden == 0 else nbr_hidden_neuron[num_hidden - 1]
		print(nbre_nn)
		hidden_layer = [{'weights':[random() for i in range(nbre_nn + 1)]} for i in range(nbr_hidden_neuron[num_hidden])] # new! : add neurons based on nbr_hidden_neuro
		network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(nbr_hidden_neuron[num_hidden]+1)]} for i in range(n_outputs)]
	network.append(output_layer)

	return network